In [ ]:
import sys
sys.path.append("..")

import random

import time
import math
import random
from io import BytesIO
from pathlib import Path
from typing import Optional, Callable, List, Tuple, Iterable, Generator, Optional, Union

from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, TensorDataset, IterableDataset
import torchvision.transforms as VT
import torchvision.transforms.functional as VF
from torchvision.utils import make_grid
from sklearn.decomposition import PCA, FactorAnalysis
from sklearn.cluster import KMeans

import PIL.Image
import PIL.ImageDraw
import plotly
import plotly.express as px
plotly.io.templates.default = "plotly_dark"
import numpy as np
import pandas as pd

from src.datasets import *
from src.util import *
from src.util.image import * 
from src.algo import Space2d, IFS
from src.datasets import *
from src.models.cnn import *
from src.util.embedding import *
from src.models.clip import ClipSingleton

In [ ]:
def plot_samples(
        iterable, 
        total: int = 32, 
        nrow: int = 8, 
        return_image: bool = False, 
        show_compression_ratio: bool = False,
        label: Optional[Callable] = None,
):
    samples = []
    labels = []
    f = ImageFilter()
    try:
        for idx, entry in enumerate(tqdm(iterable, total=total)):
            image = entry
            if isinstance(entry, (list, tuple)):
                image = entry[0]
            if image.ndim == 4:
                image = image.squeeze(0)
            samples.append(image)
            if show_compression_ratio:
                labels.append(round(f.calc_compression_ratio(image), 3))
            elif label is not None:
                labels.append(label(entry) if callable(label) else idx)
                
            if len(samples) >= total:
                break
    except KeyboardInterrupt:
        pass
    
    if labels:
        image = VF.to_pil_image(make_grid_labeled(samples, nrow=nrow, labels=labels))
    else:
        image = VF.to_pil_image(make_grid(samples, nrow=nrow, pad_value=0))
    if return_image:
        return image
    display(image)

In [ ]:
SHAPE = (3, 32, 32)
ds = RpgTileIterableDataset(shape=SHAPE, interleave=True)
ds = IterableShuffle(ds, 100_000, seed=23)

In [ ]:
images_train = []
images_test = []
for i, image in enumerate(tqdm(ds)):
    image = (image.clamp(0, 1) * 255).to(torch.uint8).unsqueeze(0)
    if i % 20 == 0:
        images_test.append(image)
    else:
        images_train.append(image)

In [ ]:
print(f"train: {len(images_train)}, test: {len(images_test)}")

In [ ]:
display(plot_samples(images_train))
display(plot_samples(images_test))

In [ ]:
shape_str = "x".join(str(s) for s in SHAPE)
torch.save(torch.concat(images_train), f"../datasets/rpg-{shape_str}-uint-train.pt")
torch.save(torch.concat(images_test), f"../datasets/rpg-{shape_str}-uint-test.pt")

In [ ]:
from experiments.datasets import mnist_dataset, fmnist_dataset

In [ ]:
def plot_resizings(image, size: int):
    def _big(x):
        return VF.to_pil_image(VF.resize(x, [s * 6 for s in x.shape[-2:]], VF.InterpolationMode.NEAREST, antialias=False))
    display(_big(image))
    display(_big(VF.resize(image, (size, size), VF.InterpolationMode.NEAREST, antialias=False)))
    display(_big(VF.resize(image, (size, size), VF.InterpolationMode.BILINEAR, antialias=True)))
    

dsm = fmnist_dataset(train=False)
plot_resizings(dsm[122][0], 32)